In [1]:
import torch
import math
from dataset import SatProblemDataSet
from models import MLP, LayerNormBasicLSTMCell, NeuroSAT
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
def compute_loss(logits, labels, parameters):
    # x = logits, z = labels
    # max(x, 0) - x * z + log(1 + exp(-abs(x)))
    predict_costs = torch.nn.functional.relu(logits) - logits * labels + torch.log1p(torch.exp(-torch.abs(logits)))
    predict_cost = torch.mean(predict_costs)
    l2_cost = torch.zeros([])
    for p in parameters:
        l2_cost += torch.sum(p ** 2) / 2
    return predict_cost + 1e-9 * l2_cost

In [3]:
n = NeuroSAT(5, 3, 3, 128)

In [4]:
ds = SatProblemDataSet("data")
dl = torch.utils.data.DataLoader(ds)

In [5]:
optim = torch.optim.Adam(n.parameters(), lr = 1e-5, weight_decay = 1e-10)

In [ ]:
n.train()
losses = []
loss_fn = torch.nn.BCELoss()
for _ in range(1):
    for prob, target in dl:
        optim.zero_grad()
        outputs = n(prob[0].float())
        torch.nn.utils.clip_grad_norm_(n.parameters(), 0.5)
        loss = compute_loss(outputs, target[0], n.parameters())
        print(outputs, target[0])
#         loss = loss_fn(outputs, target[0])
        desc = 'loss: %.4f; is_sta: %s' % (loss.item(), target[0])
        losses.append(loss.item())
        print(desc)

        loss.backward()
        optim.step()

tensor([0.0011], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6937; is_sta: tensor([0.])
tensor([0.0011], grad_fn=<AddBackward0>) tensor([1.])
loss: 0.6926; is_sta: tensor([1.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6937; is_sta: tensor([0.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([1.])
loss: 0.6926; is_sta: tensor([1.])
tensor([0.0012], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6937; is_sta: tensor([0.])
tensor([0.0011], grad_fn=<AddBackward0>) tensor([1.])
loss: 0.6926; is_sta: tensor([1.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6937; is_sta: tensor([0.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([1.])
loss: 0.6926; is_sta: tensor([1.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6936; is_sta: tensor([0.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([1.])
loss: 0.6927; is_sta: tensor([1.])
tensor([0.0010], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.6937; is_sta: tensor([0.])
tensor([0.0010], grad

In [ ]:
for _ in range(1):
    for prob, target in dl:
        optim.zero_grad()
        outputs = n(prob[0].float())
    #     print(outputs, prob[1])
        torch.nn.utils.clip_grad_norm_(n.parameters(), 0.5)
#         loss = compute_loss(outputs, target[0], n.parameters())
        print(outputs, target[0])
        loss = loss_fn(outputs, target[0])
        desc = 'loss: %.4f; ' % (loss.item())
        losses.append(loss.item())
        print(desc)

        loss.backward()
        optim.step()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot([i for i in range(len(losses))], losses)